In [1]:
# https://cran.r-project.org/web/packages/penaltyLearning/index.html

In [ ]:
# libraries
library(penaltyLearning)
library(data.table)
library(future)
library(future.apply)
library(directlabels)
library(dplyr)

In [3]:
# previous or proposed
category <- 'previous'

In [4]:
# Get the names of all folders in a specified directory
datasets <- list('cancer')

In [ ]:
for (dataset in datasets) {
    # read data
    feature.dt <- fread(paste("../../../training_data/", dataset, "/features.csv", sep = ""))
    target.dt  <- fread(paste("../../../training_data/", dataset, "/target.csv", sep = ""))
    folds.dt   <- fread(paste("../../../training_data/", dataset, "/folds.csv", sep = ""))
    
    # get number of folds
    n_folds <- dim(unique(folds.dt[,"fold"]))[1]
    
    # filter censored intervals
    target.dt  <- target.dt[!(min.log.lambda == -Inf & max.log.lambda == Inf)]
    # feature.dt <- feature.dt[sequenceID %in% target.dt$sequenceID]
    folds.dt   <- folds.dt[sequenceID %in% target.dt$sequenceID]

    # combine
    feature.dt.all <- cbind(feature.dt, folds.dt[, "fold"])
    target.dt.all  <- cbind(target.dt, folds.dt[, "fold"])
    target.dt.all <- target.dt.all %>%
    mutate(
        # Adjust min.log.lambda and max.log.lambda if they are equal
        min.log.lambda = ifelse(min.log.lambda == max.log.lambda, min.log.lambda - 0.1, min.log.lambda),
        max.log.lambda = ifelse(min.log.lambda == max.log.lambda, max.log.lambda + 0.1, max.log.lambda)
    )
    
    for (test.fold in 1:n_folds) {
        # get train and test data
        feature.mat.train <- as.matrix(feature.dt.all[feature.dt.all$fold != test.fold, -c('sequenceID', 'fold')])
        feature.mat.test  <- as.matrix(feature.dt.all[feature.dt.all$fold == test.fold, -c('sequenceID', 'fold')])
        target.mat.train  <- as.matrix(target.dt.all[target.dt.all$fold != test.fold, c("min.log.lambda", "max.log.lambda")])
        
        # train model
        fit <- IntervalRegressionCV(feature.mat = feature.mat.train, target.mat = target.mat.train)
        
        # get prediction from test set
        target.mat.pred <- fit$predict(feature.mat.test)
        colnames(target.mat.pred) <- "llda"

        # save prediction to csv
        prediction <- data.frame(as.matrix(feature.dt.all[feature.dt.all$fold == test.fold, "sequenceID"]), target.mat.pred)
        write.csv(prediction, file = paste("predictions/", paste(category, dataset, test.fold, '100', "csv", sep = "."), sep=''), row.names = FALSE)
    }
}